In [1]:
"""Trains a convolutional neural network on the MNIST dataset, then attacks it with the FGSM attack."""
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import tensorflow as tf

2024-02-13 23:06:53.471068: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-13 23:06:53.474016: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-13 23:06:53.506791: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 23:06:53.506850: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 23:06:53.508441: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [3]:
tf.compat.v1.disable_eager_execution()

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
import numpy as np

In [5]:
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier
from art.utils import load_dataset

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Read MNIST dataset
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str("mnist"))

In [7]:
# Create Keras convolutional neural network - basic architecture from Keras examples
# Source here: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=x_train.shape[1:]))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))

In [8]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
classifier = KerasClassifier(model=model, clip_values=(min_, max_))
classifier.fit(x_train, y_train, nb_epochs=5, batch_size=128)

2024-02-13 23:06:58.772411: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-02-13 23:06:58.833928: W tensorflow/c/c_api.cc:305] Operation '{name:'training/Adam/dense/bias/v/Assign' id:467 op device:{requested: '', assigned: ''} def:{{{node training/Adam/dense/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/dense/bias/v, training/Adam/dense/bias/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [10]:
# Evaluate the classifier on the test set
preds = np.argmax(classifier.predict(x_test), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
print("\nTest accuracy: %.2f%%" % (acc * 100))

/opt/conda/lib/python3.11/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2024-02-13 23:09:40.574840: W tensorflow/c/c_api.cc:305] Operation '{name:'dense_1/Softmax' id:121 op device:{requested: '', assigned: ''} def:{{{node dense_1/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](dense_1/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.



Test accuracy: 98.97%


In [11]:
# Define epsilon values
epsilon_values = [0.01, 0.1, 0.15, 0.2, 0.25, 0.3]

In [13]:
from sklearn.metrics import precision_recall_fscore_support

if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    y_test_indices = np.argmax(y_test, axis=1)
else:
    y_test_indices = y_test  # If y_test is already class indices

# Iterate over epsilon values
for epsilon in epsilon_values:
    # Craft adversarial samples with FGSM
    adv_crafter = FastGradientMethod(classifier, eps=epsilon)
    x_test_adv = adv_crafter.generate(x=x_test)

    # Evaluate the classifier on the adversarial examples
    preds = np.argmax(classifier.predict(x_test_adv), axis=1)

    # Calculate precision, recall, and F1 score
    precision_micro, recall_micro, fscore_micro, _ = precision_recall_fscore_support(
        y_test_indices, preds, average='micro')
    precision_macro, recall_macro, fscore_macro, _ = precision_recall_fscore_support(
        y_test_indices, preds, average='macro')

    # Calculate accuracy for reference
    acc = np.sum(preds == y_test_indices) / y_test_indices.shape[0]

    # Print the results for each epsilon
    print(f"Epsilon: {epsilon}")
    print(f"Test accuracy on adversarial sample: {acc * 100:.2f}%")
    print(f"Micro Precision: {precision_micro:.2f}")
    print(f"Macro Precision: {precision_macro:.2f}")
    print(f"Micro Recall: {recall_micro:.2f}")
    print(f"Macro Recall: {recall_macro:.2f}")
    print(f"Micro F1 Score: {fscore_micro:.2f}")
    print(f"Macro F1 Score: {fscore_macro:.2f}")
    print()


Epsilon: 0.01
Test accuracy on adversarial sample: 98.94%
Micro Precision: 0.99
Macro Precision: 0.99
Micro Recall: 0.99
Macro Recall: 0.99
Micro F1 Score: 0.99
Macro F1 Score: 0.99

Epsilon: 0.1
Test accuracy on adversarial sample: 88.14%
Micro Precision: 0.88
Macro Precision: 0.88
Micro Recall: 0.88
Macro Recall: 0.88
Micro F1 Score: 0.88
Macro F1 Score: 0.88

Epsilon: 0.15
Test accuracy on adversarial sample: 68.49%
Micro Precision: 0.68
Macro Precision: 0.70
Micro Recall: 0.68
Macro Recall: 0.68
Micro F1 Score: 0.68
Macro F1 Score: 0.68

Epsilon: 0.2
Test accuracy on adversarial sample: 43.51%
Micro Precision: 0.44
Macro Precision: 0.50
Micro Recall: 0.44
Macro Recall: 0.43
Micro F1 Score: 0.44
Macro F1 Score: 0.44

Epsilon: 0.25
Test accuracy on adversarial sample: 24.67%
Micro Precision: 0.25
Macro Precision: 0.34
Micro Recall: 0.25
Macro Recall: 0.25
Micro F1 Score: 0.25
Macro F1 Score: 0.26

Epsilon: 0.3
Test accuracy on adversarial sample: 16.02%
Micro Precision: 0.16
Macro Pr